In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.cassandra._
val sparkSession = SparkSession.builder().config(sc.getConf).getOrCreate()
import sparkSession.implicits._

In [3]:
sparkSession.read.textFile("file:///usr/lib/yelp_data/stopwords/stopwords_*.txt").createOrReplaceTempView("stop_words")
sparkSession.read.cassandraFormat(keyspace = "yelp", table = "review").load().createOrReplaceTempView("review")
sparkSession.read.cassandraFormat(keyspace = "yelp", table = "business").load().createOrReplaceTempView("business")

In [5]:
sparkSession.sql("""
select /*+ MAPJOIN(stop_words) */
    business_id,
    business_name,
    word,
    avg,
    total
from(
    select 
        b.business_id business_id,
        b.name business_name,
        regexp_replace(lw.w,"[^a-z]","") word,
        avg(r.stars) avg,
        sum(r.stars) total
    from review r
         inner join business b on b.business_id = r.business_id
     lateral view explode(split(lower(r.text)," ")) lw as w
    group by b.business_id,b.name,word
) T
left join stop_words sw on sw.value=T.word
where sw.value is null and T.word <> '' and total>200 
order by T.avg desc,T.total desc
"""
).show(50,false)

+------------------------------------+----------+---+-----+
|business_name                       |word      |avg|total|
+------------------------------------+----------+---+-----+
|Market Street Cafe                  |oxtail    |5.0|225  |
|Market Street Cafe                  |soup      |5.0|225  |
|Phoenicia Cafe                      |cous      |5.0|70   |
|Tacos El Gordo                      |tacos     |5.0|60   |
|The Buffet                          |buffet    |5.0|60   |
|Strolleria                          |stroller  |5.0|50   |
|Mercedes-Benz of Chandler           |car       |5.0|45   |
|Airbridge Tours                     |tour      |5.0|45   |
|Passport Photo                      |passport  |5.0|45   |
|Harumi Sushi                        |sushi     |5.0|45   |
|Fishman Lobster Clubhouse Restaurant|lobster   |5.0|45   |
|Battery Park Wine Bar               |wine      |5.0|40   |
|Burgatory                           |pick      |5.0|40   |
|Niza Lideo                          |ha